In [ ]:
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
import xarray as xr

import eradiate
from eradiate.scenes.surface import mesh_from_dem
from eradiate.units import unit_context_kernel as uck

eradiate.set_mode("mono")

In [ ]:
ds = xr.open_dataset("dem/gebco_2024/GEBCO_2024.nc").sel()
display(ds)

lon_bounds = [-5.255643817623447, -1.854804111820785]
lat_bounds = [46.98286701217718, 49.02004225968674]

lon = ds.lon.where((ds.lon > lon_bounds[0]) & (ds.lon < lon_bounds[1])).dropna(
    dim="lon"
)
lat = ds.lat.where((ds.lat > lat_bounds[0]) & (ds.lat < lat_bounds[1])).dropna(
    dim="lat"
)

brittany = ds.sel(lon=lon, lat=lat)
brittany.elevation.plot.imshow()
display(brittany)

# Spherical-shell setup

In [ ]:
da = brittany.elevation.copy()
da *= 20
da.lon.attrs["units"] = "deg"
da.lat.attrs["units"] = "deg"
mesh = mesh_from_dem(da, geometry="spherical_shell")[0]

In [ ]:
d = 750000.0
r = eradiate.constants.EARTH_RADIUS.m_as(uck.get("length"))
x = np.array([0, 0, r])

scene_dict = {
    "type": "scene",
    "illumination": {"type": "directional", "direction": [0, -1, -1]},
    # "illumination": {"type": "constant"},
    "sensor": {
        "type": "perspective",
        "to_world": mi.ScalarTransform4f.look_at(
            origin=x + np.array([0, 0, d]),
            target=x,
            up=[0, 1, 0],
        ),
        "far_clip": 1e12,
        "film": {
            "type": "hdrfilm",
            "pixel_format": "luminance",
            "component_format": "float32",
        },
    },
    "integrator": {"type": "path"},
}
scene_dict["mesh"] = mesh.instance

scene = mi.load_dict(scene_dict)
img = mi.render(scene)

plt.imshow(img)
plt.axis("off")
plt.show()
plt.close()

# Plane-parallel setup

In [ ]:
da = brittany.elevation.copy()
da *= 20
da.lon.attrs["units"] = "deg"
da.lat.attrs["units"] = "deg"
mesh = mesh_from_dem(da, geometry="plane_parallel")[0]

In [ ]:
d = 750000.0

scene_dict = {
    "type": "scene",
    "illumination": {"type": "directional", "direction": [0, -1, -1]},
    "sensor": {
        "type": "perspective",
        "to_world": mi.ScalarTransform4f.look_at(
            origin=[0, 0, d],
            target=[0, 0, 0],
            up=[0, 1, 0],
        ),
        "far_clip": 1e12,
        "film": {
            "type": "hdrfilm",
            "pixel_format": "luminance",
            "component_format": "float32",
        },
    },
    "integrator": {"type": "path"},
}
scene_dict["mesh"] = mesh.instance

scene = mi.load_dict(scene_dict)
img = mi.render(scene)

plt.imshow(img)
plt.axis("off")
plt.show()
plt.close()